In [1]:
import pickle
import d3rlpy
import pandas as pd
import numpy as np
import json

In [2]:
traindf = pd.read_parquet('gs://leo_tapas/primary/train_20240118.parquet')
validdf = pd.read_parquet('gs://leo_tapas/primary/valid_20240118.parquet')
testdf = pd.read_parquet('gs://leo_tapas/primary/test_20240118.parquet')

In [3]:
class Evaluator(d3rlpy.metrics.EvaluatorProtocol):
    def __init__(self, observations, actions, rewards, batch_size=10000):
        self.observations = observations
        self.actions = actions
        self.rewards = rewards
        self.batch_size = batch_size
    
    def _hard_metric(self, qv, r):
        return np.sum(r[np.where(qv>0)])

    def _naive_metric(self, qv, r):
        return np.sum(r[np.where((qv>0)|(r>0))])
    
    def __call__(self, algo, dataset):
        qvlist = []
        rlist = []
        for i in range(0,len(self.rewards),self.batch_size):
            qvlist.append(algo.predict_value(self.observations[i:(i+self.batch_size),:], self.actions[i:(i+self.batch_size)]))
            rlist.append(self.rewards[i:(i+self.batch_size)])
        h = self._hard_metric(qv=np.concatenate(qvlist), r=np.concatenate(rlist))
        n = self._naive_metric(qv=np.concatenate(qvlist), r=np.concatenate(rlist))
        return np.mean([h,n])

In [4]:
dataset = d3rlpy.dataset.MDPDataset(
    observations = np.stack(traindf.pca.values),
    actions = traindf.user_id_index.values,
    rewards = traindf.profit.values/1000,
    terminals = np.ones(len(traindf)))

2024-01-18 13:06.08 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('int64')], shape=[(1,)]) observation_signature=Signature(dtype=[dtype('float64')], shape=[(256,)]) reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)])
2024-01-18 13:06.08 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.DISCRETE: 2>
2024-01-18 13:06.18 [info     ] Action size has been automatically determined. action_size=144881


In [5]:
valid_eval = Evaluator(
    observations = np.stack(validdf.pca.values),
    actions = validdf.user_id_index.values,
    rewards = validdf.profit.values/1000)
test_eval = Evaluator(
    observations = np.stack(testdf.pca.values),
    actions = testdf.user_id_index.values,
    rewards = testdf.profit.values/1000)

In [6]:
batch_size = 128
valid_eval = Evaluator(
    observations = np.stack(validdf.pca.values),
    actions = validdf.user_id_index.values,
    rewards = validdf.profit.values/1000,
    batch_size = batch_size)
test_eval = Evaluator(
    observations = np.stack(testdf.pca.values),
    actions = testdf.user_id_index.values,
    rewards = testdf.profit.values/1000,
    batch_size = batch_size)
cql = d3rlpy.algos.DiscreteCQLConfig(batch_size=batch_size).create(device='cuda:0')

In [7]:
cql.fit(
    dataset = dataset, 
    n_steps = int(1e+5),
    n_steps_per_epoch = int(1e+4),
    evaluators = {
        "valid": valid_eval,
        "test": test_eval,
})

2024-01-18 13:06.25 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float64')], shape=[(256,)]), action_signature=Signature(dtype=[dtype('int64')], shape=[(1,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.DISCRETE: 2>, action_size=144881)
2024-01-18 13:06.25 [info     ] Directory is created at d3rlpy_logs/DiscreteCQL_20240118130625
2024-01-18 13:06.25 [debug    ] Building models...            
2024-01-18 13:06.27 [debug    ] Models have been built.       
2024-01-18 13:06.27 [info     ] Parameters                     params={'observation_shape': [256], 'action_size': 144881, 'config': {'type': 'discrete_cql', 'params': {'batch_size': 128, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'learning_rate': 6.25e-05, 'optim_factory': {'type': 'adam', 'params

Epoch 1/10:   0%|          | 0/10000 [00:00<?, ?it/s]

2024-01-18 13:16.10 [info     ] DiscreteCQL_20240118130625: epoch=1 step=10000 epoch=1 metrics={'time_sample_batch': 0.006409164595603943, 'time_algorithm_update': 0.048495398688316343, 'loss': 13.165841132354736, 'td_loss': 2.00659384329319, 'conservative_loss': 11.159247290420533, 'time_step': 0.055090593957901, 'valid': 161712.08800000002, 'test': 352847.8805} step=10000
2024-01-18 13:16.11 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20240118130625/model_10000.d3


Epoch 2/10:   0%|          | 0/10000 [00:00<?, ?it/s]

2024-01-18 13:25.51 [info     ] DiscreteCQL_20240118130625: epoch=2 step=20000 epoch=2 metrics={'time_sample_batch': 0.006378501033782959, 'time_algorithm_update': 0.048288755631446835, 'loss': 12.911194874191285, 'td_loss': 2.032807874971628, 'conservative_loss': 10.878386997699737, 'time_step': 0.05485282573699951, 'valid': 163289.915, 'test': 356791.68399999995} step=20000
2024-01-18 13:25.52 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20240118130625/model_20000.d3


...

Epoch 9/10:   0%|          | 0/10000 [00:00<?, ?it/s]

2024-01-18 14:33.28 [info     ] DiscreteCQL_20240118130625: epoch=9 step=90000 epoch=9 metrics={'time_sample_batch': 0.0062477138996124264, 'time_algorithm_update': 0.04823018491268158, 'loss': 12.584909728622437, 'td_loss': 2.083630687242746, 'conservative_loss': 10.501279044437409, 'time_step': 0.05466206026077271, 'valid': 169820.65399999998, 'test': 367017.2225000001} step=90000
2024-01-18 14:33.30 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20240118130625/model_90000.d3


Epoch 10/10:   0%|          | 0/10000 [00:00<?, ?it/s]

2024-01-18 14:43.07 [info     ] DiscreteCQL_20240118130625: epoch=10 step=100000 epoch=10 metrics={'time_sample_batch': 0.006192622065544128, 'time_algorithm_update': 0.04820101864337921, 'loss': 12.58577006816864, 'td_loss': 2.09108464987278, 'conservative_loss': 10.494685424423217, 'time_step': 0.05457674689292908, 'valid': 171119.87900000002, 'test': 369410.23850000004} step=100000
2024-01-18 14:43.08 [info     ] Model parameters are saved to d3rlpy_logs/DiscreteCQL_20240118130625/model_100000.d3


[(1,
  {'time_sample_batch': 0.006409164595603943,
   'time_algorithm_update': 0.048495398688316343,
   'loss': 13.165841132354736,
   'td_loss': 2.00659384329319,
   'conservative_loss': 11.159247290420533,
   'time_step': 0.055090593957901,
   'valid': 161712.08800000002,
   'test': 352847.8805}),
 (2,
  {'time_sample_batch': 0.006378501033782959,
   'time_algorithm_update': 0.048288755631446835,
   'loss': 12.911194874191285,
   'td_loss': 2.032807874971628,
   'conservative_loss': 10.878386997699737,
   'time_step': 0.05485282573699951,
   'valid': 163289.915,
   'test': 356791.68399999995}),
 (3,
  {'time_sample_batch': 0.006408561205863953,
   'time_algorithm_update': 0.04831157054901123,
   'loss': 12.784132319164277,
   'td_loss': 2.035749557322264,
   'conservative_loss': 10.748382761192321,
   'time_step': 0.05490975980758667,
   'valid': 163866.585,
   'test': 357164.657}),
 (4,
  {'time_sample_batch': 0.0063162554740905765,
   'time_algorithm_update': 0.0482660311460495,
  